# Even functions

Read through the following tutorial and fill in all the places where there is something missing (___)

A function $f : [−a, a] \rightarrow \mathbb{R}$ is called even if
$f (x) = f (−x)$.

Below we try to learn an even function. First import the relevant packages. 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

import plotly.express as px
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

## Data

Here we generate some data of an even function. 

In [2]:
def f(x):
    return 2-np.cos(x) + (np.cos(3*x))**3-(np.cos(5*x))**5

In [73]:
rng = np.random.RandomState(0)

n_samples = 100
x = rng.uniform(low=-5, high=5, size=n_samples)

# Sample y=f(x) + noise
y = f(x) + rng.normal(loc=0, scale=0.25, size=n_samples)

In [74]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x, y, test_size=.3, random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size=.5, random_state=0)

In [75]:
fig = px.scatter(x=x_train, y=y_train)
x_values = np.linspace(-5, 5, 200)
fig.add_trace(go.Scatter(x=x_values, y=f(x_values), showlegend=False))
fig.show()

Just for reference, we compute the best possible error, i.e. the validation error we would get if we knew the correct function. 

In [76]:
best_pred = f(x_val)
best_error = mean_squared_error(y_val, best_pred, squared=False)
print(best_error)

0.2608052215422928


/Users/martineh/Library/Caches/pypoetry/virtualenvs/gdl-pO0IvMrQ-py3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



## Non-invariant solution

We first fit a polynomial regression of degree 12. 

In [77]:
model1 = make_pipeline(PolynomialFeatures(degree=12), LinearRegression())

In [78]:
model1.fit(x_train.reshape(-1, 1), y_train)
pred = model1.predict(x_val.reshape(-1, 1))
ni_rmse = mean_squared_error(y_val, pred, squared=False)

print("true:", best_error, ", 12th order polynomial:", ni_rmse)

true: 0.2608052215422928 , 12th order polynomial: 0.8249478153821688


/Users/martineh/Library/Caches/pypoetry/virtualenvs/gdl-pO0IvMrQ-py3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



In [79]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=y_train, mode='markers', showlegend=False))
fig.add_trace(go.Scatter(x=x_values, y=f(x_values), name='true function'))  
fig.add_trace(go.Scatter(x=x_values, 
                             y=model1.predict(x_values.reshape(-1, 1)), 
                             name='12poly'))
fig.show()

## Smoothing predictions

A first possibility to get an invariant model is to smooth predictions. That is, we use the original model for training, but when predicting, we use the smooth version. 

In [80]:
# We apply smoothening operator to our model.
class EvenModel(BaseEstimator):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def fit(self, X, y):
        self.model.fit(X, y)
        
    def predict(self, X):
        # NOTE: Use that f(x) = f(-x) ?
        predictions = np.array([self.model.predict(X), self.model.predict(-X)])
        return np.mean(predictions, axis=0)

In [81]:
# We only consider even function
model2 = EvenModel(model1)

In [82]:
# We get an improved result for our second approach
model2.fit(x_train.reshape(-1, 1), y_train)
pred2 = model2.predict(x_val.reshape(-1, 1))
ip_rmse = mean_squared_error(y_val, pred2, squared=False)

print("true:",best_error, ", 12th order polynomial:", ip_rmse)

true: 0.2608052215422928 , 12th order polynomial: 0.7383242354750466


/Users/martineh/Library/Caches/pypoetry/virtualenvs/gdl-pO0IvMrQ-py3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=y_train, mode='markers', showlegend=False))
fig.add_trace(go.Scatter(x=x_values, y=f(x_values), name='true function'))
fig.add_trace(go.Scatter(x=x_values, 
                            y=model2.predict(x_values.reshape(-1, 1)), 
                            name='12poly-Even'))
fig.show()

## Smoothing training

Another possibility is to already take into account the fact that the function should be even in training and use the smmothing operator also for training. 

In [84]:
class EvenPolyNN(nn.Module):
    def __init__(self, deg=12):
        super().__init__()
        self.deg = deg
        self.layer = nn.Linear(self.deg, 1)

    def forward(self, x):
        positive = torch.cat([x.pow(k) for k in range(1, self.deg+1)], dim = 1)
        negative = torch.cat([(-x).pow(k) for k in range(1, self.deg+1)], dim = 1)
        out = self.layer(positive) + self.layer(negative)
        return out.flatten()/2

In [85]:
trainset = TensorDataset(torch.Tensor(x_train.reshape(-1, 1)), torch.Tensor(y_train))
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)

valset = TensorDataset(torch.Tensor(x_val.reshape(-1, 1)), torch.Tensor(y_val))
valloader = DataLoader(valset, batch_size=16, shuffle=False)

In [86]:
model3 = EvenPolyNN(12) 
model3.layer.bias.data = torch.Tensor(np.array([model1[1].intercept_]))
model3.layer.weight.data = torch.Tensor(model1[1].coef_[1:].reshape(1, -1))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model3.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-8, weight_decay=1e-8)

model.train()

EvenPolyNN(
  (layer): Linear(in_features=12, out_features=1, bias=True)
)

In [87]:
# training
n_epochs = 100

for epoch in range(n_epochs):
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.mse_loss(outputs, labels)
        loss.backward()
        optimizer.step()

In [88]:
with torch.no_grad():
    pred3 = np.array([np.array(model(x)) for x, _ in valloader]).flatten()

In [89]:
mean_squared_error(y_val, pred3, squared=False)

/Users/martineh/Library/Caches/pypoetry/virtualenvs/gdl-pO0IvMrQ-py3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



0.7342533745934376

In [90]:
dset = TensorDataset(torch.Tensor(x_values.reshape(-1, 1)))
dloader = DataLoader(dset, batch_size=16, shuffle=False)

with torch.no_grad():
    out = np.hstack([np.array(model(x)) for x, in dloader]).flatten()


In [91]:
it_rmse = mean_squared_error(y_val, pred3, squared=False)
print("true:", best_error, ", 12th order polynomial:", it_rmse)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=y_train, mode='markers', showlegend=False))
fig.add_trace(go.Scatter(x=x_values, y=f(x_values), name='true function'))
fig.add_trace(go.Scatter(x=x_values, 
                         y=out, 
                         name='trained 12poly-Even'))
fig.show()

true: 0.2608052215422928 , 12th order polynomial: 0.7342533745934376


/Users/martineh/Library/Caches/pypoetry/virtualenvs/gdl-pO0IvMrQ-py3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



In [92]:
print('Non-invariant RMSE:', ni_rmse)
print('Invariant prediction RMSE:', ip_rmse)
print('Invariant training RMSE:', it_rmse)

Non-invariant RMSE: 0.8249478153821688
Invariant prediction RMSE: 0.7383242354750466
Invariant training RMSE: 0.7342533745934376


What did you learn from this?

___

The prediction was slightly improved by introducing smoothing predictions. However, it was nearly no difference between introducing it during training or durnig prediction. This is because the problem we are considering is not solvable with a 12th order polynomial, even if we enforce invariance.